<a href="https://colab.research.google.com/github/Mariahsth/agente-ia-gemini/blob/main/AgentesIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

langchain e langchain do googleAI-> framework par criar agentes de IA  

google-generativeai -> SDK oficial do google, que vai fazer a interação com os modelos

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00


## Conectar ao Gemini
(buscar API key no google AI Studio e colocar na variável de ambiente)

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

## Preparar modelo, temperatura e API KEY

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
# teste
resp_test=llm.invoke("Quem é você?")
print(resp_test.content)

## Definir o contexto do agente
Criar PROMPT para fazer triagem entre:  
1- Responder usando RAG  
2- Abrir ticket  
3- Pedir mais informações  

In [ ]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

Testar prompt  
-> Instalar biblioteca que auxilia a dar uma saída mais estruturada, como pydantic  
-> Instalar biblioteca de dicionário de dados, como o typing

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao:Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str]= Field(default_factory=list)

In [ ]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

Estruturar mensagens enviadas pelo modelo, diferenciando quando é mensagem do sistema e quando é mensagem humama

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem (mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])
  return saida.model_dump()


In [ ]:
testes = ["Posso reembolsar a internet?",
          "Posso ter mais do que 5 dias remoto?",
          "Posso reembolsar cursos ou treinamentos da alura?",
          "Quantas capivaras tem no Rio Pinheiros?"
          ]

for msg_testes in testes:
  print(f"Pergunta: {msg_testes}\n -> Resposta: {triagem(msg_testes)}\n" )

## Configurando RAG

#### Instalações:
langchain_community ->  vetorização, busca semântica, etc..  
faiss-cpu ->  ver similaridade entre os pedaços de texto  
langchain-text-splitter ->  quebrar texto em vários pedaços  
pymupdf -> ler pdf

In [ ]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf


Carregando arquivos da empresa

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs=[]

for n in Path("/content/").glob("*.pdf"):
  try:
    loader = PyMuPDFLoader(str(n))
    docs.extend(loader.load())
    print(f"Carregado arquivo {n.name} com sucesso")
  except Exception as e:
    print(f"Erro ao carregar arquivo {n.name}:{e}")

print(f"Total de documentos carregados: {len(docs)}")

Dividindo os pdfs em partes menores (chunks)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap =30)

chunks = splitter.split_documents(docs)

In [ ]:
chunks

In [ ]:
for chunk in chunks:
  print(chunk)
  print("\n-------------------------\n")

## Embeddings

Transformando os chunks em vetores (para saber como cada um se relaciona)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore=FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold": 0.3, "k":4})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
        ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain= create_stuff_documents_chain(llm_triagem, prompt_rag)

In [ ]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [ ]:
def perguntar_politica_RAG(pergunta:str) -> Dict:
  docs_relacionados=retriever.invoke(pergunta)

  if not docs_relacionados:
    return {"answer" : "Não sei.",
            "citacoes": [],
            "contexto_encontrado":False}
  answer = document_chain.invoke({"input": pergunta,
                                  "context":docs_relacionados})

  txt = (answer or "").strip()

  if txt.rstrip(".!?") == "Não sei":
        return {"answer" : "Não sei.",
            "citacoes": [],
            "contexto_encontrado":False}

  return {"answer" : txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado":True}


In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da alura?",
          "Quantas capivaras tem no Rio Pinheiros?"
          ]


In [ ]:
for msg_testes in testes:
  resposta = perguntar_politica_RAG(msg_testes)
  print(f"Pergunta: {msg_testes}\n -> Resposta: {resposta['answer']}" )
  if resposta['contexto_encontrado']:
    print(f" -> Citações: \n" )
    for c in resposta['citacoes']:
      print(f"- Documento: {c['documento']}, Página: {c['pagina']} \n Trecho: {c['trecho']}")
  print("-----------------------------------\n")



instalar langgraph -> utilizando grafos para estruturar o agente

In [ ]:
!pip install -q --upgrade langgraph

In [ ]:
from typing import TypedDict, Optional

class AgentState(TypedDict, total=False):
  pergunta: str
  triagem: Dict
  resposta: Optional[str]
  citacoes:List[dict]
  rag_sucesso: bool
  acao_final: str

In [ ]:
def node_triagem(state: AgentState) -> AgentState:
  print("Excecutando nó de triagem...")
  return {"triagem": triagem(state["pergunta"])}



In [ ]:
def node_auto_resolver(state: AgentState) -> AgentState:
  print("Excecutando nó de auto resolver...")
  resposta_rag=perguntar_politica_RAG(state["pergunta"])

  update:AgentState = {
      "resposta":resposta_rag["answer"],
      "citacoes":resposta_rag.get('citacoes', []),
      "rag_sucesso":resposta_rag["contexto_encontrado"],
  }

  if resposta_rag["contexto_encontrado"]:
    update["acao_final"] = "AUTO_RESOLVER"

  return update

In [ ]:
def node_pedir_info(state: AgentState) -> AgentState:
  print("Excecutando nó de pedir info...")

  faltantes = state["triagem"].get("campos_faltantes", [])

  if faltantes:
    detalhe = ",".join(faltantes)
  else:
    detalhe = "Tema e contexto específico"

  return {"resposta": f"Para avançar preciso que detalhe: {detalhe}",
          "citacoes": [],
          "acao_final": "PEDIR_INFO"}

In [ ]:
def node_abrir_chamado(state: AgentState) -> AgentState:
  print("Excecutando nó de pedir chamado...")

  triagem=state["triagem"]

  return{
      "resposta":f"Abrindo chamado com urgência {triagem['urgencia']}. Descrição: {state['pergunta'][:140]}",
      "citacoes":[],
      "acao_final":"ABRIR_CHAMADO"
  }

In [ ]:
KEYWORDS_ABRIR_TICKET = ["aprovação", "exceção", "liberação", "abrir ticket", "abrir chamado", "acesso especial"]

def decidir_pos_triagem(state: AgentState) -> str:
  print("Decidindo após a triagem...")
  decisao=state["triagem"]["decisao"]

  if decisao == "AUTO_RESOLVER": return "auto"
  if decisao == "PEDIR_INFO": return "info"
  if decisao == "ABRIR_CHAMADO": return "chamado"

In [ ]:
def decidir_pos_auto_resolver(state: AgentState) -> str:
  print("Decidindo após o auto resolver...")

  if state.get("rag_sucesso"):
    print("Rag com sucesso, finalizando o fluxo")
    return "ok"

  state_da_pergunta = (state["pergunta"] or "").lower()

  if any(k in state_da_pergunta for k in KEYWORDS_ABRIR_TICKET):
    print("Rag falhou, mas foram encontradas keywords de abertura de ticket, abrindo...")
    return "chamado"

  print("Rag falhou, sem keywords, vou pedir mais informações...")
  return "info"


In [ ]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(AgentState)

workflow.add_node("triagem", node_triagem)
workflow.add_node("auto_resolver", node_auto_resolver)
workflow.add_node("pedir_info", node_pedir_info)
workflow.add_node("abrir_chamado", node_abrir_chamado)


workflow.add_edge(START, "triagem")
workflow.add_conditional_edges("triagem", decidir_pos_triagem,
 {"auto": "auto_resolver",
  "info":"pedir_info",
  "chamado":"abrir_chamado"
  })
workflow.add_conditional_edges("auto_resolver", decidir_pos_auto_resolver,
 {
  "info":"pedir_info",
  "chamado":"abrir_chamado",
  "ok":END
  })
workflow.add_edge("pedir_info", END)
workflow.add_edge("abrir_chamado", END)

grafo=workflow.compile()

In [ ]:
from IPython.display import display, Image

graph_bytes = grafo.get_graph().draw_mermaid_png()
display(Image(graph_bytes))

In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da alura?",
          "Tem como reembolsar certificações do Google Cloud?",
          "Posso obter o Google Gemini de graça?",
          "Qual é a palavra chave de hoje?",
          "Quantas capivaras tem no Rio Pinheiros?"
          ]

In [ ]:
for msg_teste in testes:
  resposta_final= grafo.invoke({"pergunta":msg_teste})

  triag = resposta_final.get("triagem", {})
  print(f"PERGUNTA: {msg_teste}")
  print(f"DECISÃO: {triag.get('decisao')} | URGÊNCIA: {triag.get('urgencia')} | AÇÃO FINAL: {resposta_final.get('acao_final')}")
  print(f"RESPOSTA: {resposta_final.get('resposta')}")
  if resposta_final.get('citacoes'):
    print("CITAÇÕES: ")
    for citacao in resposta_final.get('citacoes'):
       print(f"- Documento: {citacao['documento']}, Página: {citacao['pagina']} \n Trecho: {citacao['trecho']}")
  print("-----------------------------------\n")
